In [100]:
import time
import os

import pandas as pd
import numpy as np
import librosa

from pydub import AudioSegment
from pydub.utils import mediainfo

from monitoring import *

In [101]:
taxonomy = pd.read_csv('data/taxonomy.csv')
df = pd.read_csv('data/train.csv')

In [102]:
taxonomy

,primary_label,inat_taxon_id,scientific_name,common_name,class_name
0,1139490,1139490,Ragoniella pulchella,Ragoniella pulchella,Insecta
1,1192948,1192948,Oxyprora surinamensis,Oxyprora surinamensis,Insecta
2,1194042,1194042,Copiphora colombiae,Copiphora colombiae,Insecta
3,126247,126247,Leptodactylus insularum,Spotted Foam-nest Frog,Amphibia
4,1346504,1346504,Neoconocephalus brachypterus,Neoconocephalus brachypterus,Insecta
...,...,...,...,...,...
201,yehcar1,1432779,Milvago chimachima,Yellow-headed Caracara,Aves
202,yelori1,9352,Icterus nigrogularis,Yellow Oriole,Aves
203,yeofly1,16567,Tolmomyias sulphurescens,Yellow-olive Flycatcher,Aves
204,yercac1,10359,Cacicus cela,Yellow-rumped Cacique,Aves


In [103]:
print(df.columns)
df

Index(['primary_label', 'secondary_labels', 'type', 'filename', 'collection',
       'rating', 'url', 'latitude', 'longitude', 'scientific_name',
       'common_name', 'author', 'license'],
      dtype='object')


,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28559,ywcpar,[''],[''],ywcpar/iNat77392.ogg,iNat,0.0,https://static.inaturalist.org/sounds/77392.m4a,7.6921,-80.3379,Amazona ochrocephala,Yellow-crowned Parrot,Yennifer Alfaro,cc-by-nc 4.0
28560,ywcpar,[''],[''],ywcpar/iNat78624.ogg,iNat,0.0,https://static.inaturalist.org/sounds/78624.wav,8.9918,-79.4877,Amazona ochrocephala,Yellow-crowned Parrot,Evan Centanni,cc-by-nc-sa 4.0
28561,ywcpar,[''],[''],ywcpar/iNat789234.ogg,iNat,0.0,https://static.inaturalist.org/sounds/789234.wav,9.2316,-70.2041,Amazona ochrocephala,Yellow-crowned Parrot,Henrry,cc-by 4.0
28562,ywcpar,[''],[''],ywcpar/iNat819873.ogg,iNat,0.0,https://static.inaturalist.org/sounds/819873.mp3,10.5838,-66.8545,Amazona ochrocephala,Yellow-crowned Parrot,Alejandro Luy,cc-by-nc 4.0


From here i am going to build a basic function that opens a given file from a filename and extracts information about it from the metadata of the file.

In [104]:
def get_audio_metadata(filename: str) -> dict:
    """
    Extract audio metadata from a file.

    Args:
        filename (str): Path to the audio file as listed in the dataframe.

    Returns:
        dict: A dictionary containing the following metadata:
            - sample_rate (str): The sample rate of the audio file.
            - bit_rate (str): The bit rate of the audio file.
            - sample_fmt (str): The sample format of the audio file.
            - duration (int): The duration of the audio file in milliseconds.
            - num_channels (int): The number of channels in the audio file.
            - file_codec (str): The codec used in the audio file.
            - file_codec_long_name (str): The long name of the codec.
            - array_size (int): The size of the audio sample array for a single channel.
    """
    filename = 'data/train_audio/' + filename
    info = mediainfo(filename)
    sound = AudioSegment.from_file(filename)
    array = np.array(sound.get_array_of_samples())
    meta_data = {
        'sample_rate': info['sample_rate'],
        'bit_rate': info['bit_rate'],
        'sample_fmt': info['sample_fmt'],
        'duration': len(sound),
        'num_channels': sound.channels,
        'file_codec': info['codec_name'],
        'file_codec_long_name': info['codec_long_name'],
        'array_size': len(array)
    }
    return meta_data


In [105]:

get_audio_metadata(df.iloc[0]['filename'])

{'sample_rate': '32000',
 'bit_rate': '62385',
 'sample_fmt': 'fltp',
 'duration': 98853,
 'num_channels': 1,
 'file_codec': 'vorbis',
 'file_codec_long_name': 'Vorbis',
 'array_size': 3163308}

In [106]:
display = None
for i in range(10):
    time.sleep(0.1)
    display = print_progress_bar(i+1, 10, "test", display_handler=display)

In [107]:
columns = df.columns.tolist()
columns.append('sample_rate')
columns.append('bit_rate')
columns.append('sample_fmt')
columns.append('duration')
columns.append('num_channels')
columns.append('file_codec')
columns.append('file_codec_long_name')
columns.append('array_size')
new_df = pd.DataFrame(columns=columns)

length = len(df)
start = time.time()
display = None


if not os.path.exists('exports/train_metadata.csv'):
    for index, row in df.iterrows():
        display = print_progress_bar(index, length, calculate_time_remaining(start, index, length), display)
        filename = row['filename']
        meta_data = get_audio_metadata(filename)
        new_row = {
            'filename': filename,
            'primary_label': row['primary_label'],
            'secondary_labels': row['secondary_labels'],
            'type': row['type'],
            'collection': row['collection'],
            'rating': row['rating'],
            'url': row['url'],
            'latitude': row['latitude'],
            'longitude': row['longitude'],
            'scientific_name': row['scientific_name'],
            'common_name': row['common_name'],
            'author': row['author'],
            'license': row['license'],
            'sample_rate': meta_data['sample_rate'],
            'bit_rate': meta_data['bit_rate'],
            'sample_fmt': meta_data['sample_fmt'],
            'duration': meta_data['duration'],
            'num_channels': meta_data['num_channels'],
            'file_codec': meta_data['file_codec'],
            'file_codec_long_name': meta_data['file_codec_long_name'],
            'array_size': meta_data['array_size']
        }
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
    new_df.to_csv('exports/train_metadata.csv', index=False)
else:
    new_df = pd.read_csv('exports/train_metadata.csv')

In [108]:
new_df

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,...,author,license,sample_rate,bit_rate,sample_fmt,duration,num_channels,file_codec,file_codec_long_name,array_size
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,...,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62385,fltp,98853,1,vorbis,Vorbis,3163308
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,...,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62574,fltp,96538,1,vorbis,Vorbis,3089207
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,...,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,61940,fltp,116600,1,vorbis,Vorbis,3731194
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,...,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62444,fltp,105446,1,vorbis,Vorbis,3374282
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,...,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62875,fltp,103631,1,vorbis,Vorbis,3316207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28559,ywcpar,[''],[''],ywcpar/iNat77392.ogg,iNat,0.0,https://static.inaturalist.org/sounds/77392.m4a,7.6921,-80.3379,Amazona ochrocephala,...,Yennifer Alfaro,cc-by-nc 4.0,32000,69806,fltp,3392,1,vorbis,Vorbis,108544
28560,ywcpar,[''],[''],ywcpar/iNat78624.ogg,iNat,0.0,https://static.inaturalist.org/sounds/78624.wav,8.9918,-79.4877,Amazona ochrocephala,...,Evan Centanni,cc-by-nc-sa 4.0,32000,64741,fltp,7660,1,vorbis,Vorbis,245110
28561,ywcpar,[''],[''],ywcpar/iNat789234.ogg,iNat,0.0,https://static.inaturalist.org/sounds/789234.wav,9.2316,-70.2041,Amazona ochrocephala,...,Henrry,cc-by 4.0,32000,65910,fltp,7667,1,vorbis,Vorbis,245333
28562,ywcpar,[''],[''],ywcpar/iNat819873.ogg,iNat,0.0,https://static.inaturalist.org/sounds/819873.mp3,10.5838,-66.8545,Amazona ochrocephala,...,Alejandro Luy,cc-by-nc 4.0,32000,58547,fltp,27037,1,vorbis,Vorbis,865176


In [109]:
new_df['sample_rate'].value_counts()

sample_rate
32000    28564
Name: count, dtype: int64

In [110]:
new_df['bit_rate'].value_counts()

bit_rate
62880    12
63689    12
63738    11
61896    11
62588    11
         ..
56662     1
58749     1
55016     1
69805     1
59390     1
Name: count, Length: 12464, dtype: int64

In [111]:
new_df['sample_fmt'].value_counts()

sample_fmt
fltp    28564
Name: count, dtype: int64

In [112]:
new_df['file_codec'].value_counts()

file_codec
vorbis    28564
Name: count, dtype: int64

In [113]:
new_df['file_codec_long_name'].value_counts()

file_codec_long_name
Vorbis    28564
Name: count, dtype: int64